### Run cell below to play Boxy!
##### Requires PyGame Module

In [ ]:
import pygame

pygame.init()

# Standard modules
import math, copy, numpy as np
from random import randint
#from time import sleep

# Custom 
from Box import Box, resolve_fall
from Super_Classes import Body
from Gettables import Fruit
from Status import Status
from Player import Player
from Constants import display_size, S
from Level import Level
from Make_Sounds import ouch_sound, thud_sound



gameDisplay = pygame.display.set_mode(display_size)
pygame.display.set_caption('Game!')

black = (0,0,0)
white = (255,255,255)
sky = (205, 230, 247)
clock = pygame.time.Clock()
background_speed = 7.0/8.0

bounce_vel = 6.0*S


# slam shifts for flop hits
shifts = [np.array([randint(-10,10)/5,randint(-10,10)/5]) for _ in range(5)]
shifts.append(np.array([0,0]))
  
# set keys to false
left_key = False
right_key = False
up_key = False
down_key = False
crouch_key = False
jump_hold = False

# Object to determine what to draw on screen
screen = Body([0,0],display_size,corporeal=True,solid=False,velocity=[0,0])   

# Object holding all level Bodies and scenery
level = Level(num=0)
    
death_delay = 60 # time after death level keeps tickin'
crashed = death_delay
jump_timer = 0
ticker = -1

while True:
    
    if crashed or not screen.overlap(character): # character off screen = dead!
        if crashed == death_delay:
            character= Player(level.player_start)
            crashed = False
            level.reset()
            character.current_status = Status(lives = 14, fruit = 85, boxes = 0)
        else:    
            crashed += 1
            
        if crashed == 2:
            ouch_sound()

            
##############################################################################
    # Handle key inputs
    
    jump_timer -=1
    jump_key = False
    attack_key = False
    flop_key = False

    for event in pygame.event.get(): ### THIS IS NEEDED OR IT WILL CRASH

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                left_key = True
            elif event.key == pygame.K_RIGHT:
                right_key = True
                
            elif event.key == pygame.K_UP:
                up_key = True
            elif event.key == pygame.K_DOWN:
                down_key = True
                
            elif event.key == pygame.K_c:
                crouch_key = True
                flop_key = True
        
            elif event.key == pygame.K_x:
                jump_key = True
                jump_timer = character.jump_anticipation
                jump_hold = True
            
            elif event.key == pygame.K_z:
                attack_key = True
   
            elif event.key == pygame.K_q:
                if not crashed:
                    crashed = True
                
        elif event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT:
                left_key = False
            elif event.key == pygame.K_RIGHT:
                right_key = False
            elif event.key == pygame.K_c:
                crouch_key = False
            
            elif event.key == pygame.K_UP:
                up_key = False
            elif event.key == pygame.K_DOWN:
                down_key = False
            elif event.key == pygame.K_x:
                jump_hold = False

##############################################################################    
    # Move every non-player object and resolve their interactions
    
    level.move_objects(land_sound = thud_sound) # input is sound boxes make when they hit the ground
    

##############################################################################
    # Evolve character state, move character, and resolve character interactions

    if not crashed: # if character is dead, don't bother with this stuff until it's regenerated
          
        # determine character's state and velocity
        character.evolve(right_key-left_key,crouch_key,jump_key,attack_key,flop_key,jump_hold)
        character.move()
          
        # sort box_list so that closest bodies interact first: prevents e.g. diagonally breaking boxes
        level.box_list = sorted(level.box_list, key= lambda x: sum((character.pos-x.pos)**2), reverse=False)
        
       # squeeze = [[False,False],[False,False]]
     
        # all objects interact with the player (destroy boxes, get fruit, land on platforms, etc)
        for i in level.box_list+level.platform_list+level.gettable_list:
            i.interact(character)
            
        # some of these interactions might have killed ya.    
        crashed = (character.protection == -1) and not character.invulnerable 
        
##############################################################################
    # Prepare canvas and draw visuals
    
    # move all non-corporeal objects to the foreground
    for body_list in [level.box_list, level.gettable_list]:
        for bod in body_list[::-1]: # need to go in reverse else removal of two objects doesn't work
            if not bod.corporeal:
                body_list.remove(bod)
                level.foreground_list.append(bod)
            
    
    # Earthquake maker!
    if ticker == -1 and character.flopping == (character.flop_stun-1):
        ticker = len(shifts)-2  
        
    elif ticker>=0:
        ticker -=1
    
    
    # fill display with white and set center position
    gameDisplay.fill(sky)
    screen.pos = np.array([character.pos[0]-display_size[0]/2,0])
    
   
    # draw all bodies
    big_list = [level.background_list, level.platform_list, level.box_list, level.gettable_list, level.foreground_list]

    if not crashed:
        big_list.insert(len(level.background_list),[character])
  
    for bod in level.scenery:
        if abs((screen.pos[0]-bod.pos[0])*(1.0-background_speed))<(screen.size[0]+bod.size[0]):
            bod.draw(gameDisplay,screen.pos - [(character.pos[0]-bod.pos[0])*background_speed,0])
    
    for small_list in big_list:
        for bod in small_list:
            if ticker>=0:
                bod.visual_shift(shifts[ticker])
            if screen.overlap(bod):
                bod.draw(gameDisplay,screen.pos)
            if ticker>=0:
                bod.visual_shift(-shifts[ticker])

    # draw counters and icons at top
    character.current_status.draw(gameDisplay)
    
    
  #  sleep(0.1)
    pygame.display.update()
    clock.tick(45)

    
pygame.quit()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [6]:
level.master_platform_list[3].on_me

To Do:

Add spring boxes (metal/wood, one class that adds a shape of an arrow, and that's it, rest will be handled outside)

Add Tnt destruct sequence

Add protection box (and following sprite?)

Level saving

Game structure, saving, scores, etc.

### Bouncing off boxes is not currently configured so that jumping adds to the bounce.

### Squeezing is not currently configured at all (to kill player) (is this needed?)
#### Definitely needed: falling non-breakable boxes do not kill player

Known bugs:
falling stacks of boxes ON a moving platform results in slight overlap (but nothing functionally problematic)

Credits:
“Sound effects obtained from zapsplat.com and freesound.org"
"Idea inspired by but unrelated to Crash Bandicoot"
